In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
np.finfo(np.float)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore', category = DeprecationWarning)
import nltk
from nltk import word_tokenize
from datetime import datetime
import scipy.stats.distributions as dist

In [2]:
pd.set_option('display.max_columns', None)

file = pd.read_csv(r'C:\Users\xuanx\Desktop\Steam_Reviews\steam_reviews.csv')

In [3]:
file['language'].value_counts()[:5]

english      9635437
schinese     3764967
russian      2348900
brazilian     837524
spanish       813320
Name: language, dtype: int64

In [4]:
#pick out english reviews only
df = file[file['language'] == 'english']

In [5]:
df['recommended'].value_counts()

True     8584444
False    1050993
Name: recommended, dtype: int64

In [6]:
#Checking review column for null values and removing them.
print(df.review.isnull().sum())
df1 = df[df.review.isnull()== False]

16750


In [7]:
df1

,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,0.000000,0,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
5,5,292030,The Witcher 3: Wild Hunt,85184171,english,"good story, good graphics. lots to do.",1611379264,1611379264,True,0,0,0.000000,0,True,False,False,76561198170193529,11,1,823.0,823.0,823.0,1.611379e+09
6,6,292030,The Witcher 3: Wild Hunt,85184064,english,"dis gud,",1611379091,1611379091,True,0,0,0.000000,0,True,False,False,76561198119302812,27,2,4192.0,3398.0,4192.0,1.611352e+09
18,18,292030,The Witcher 3: Wild Hunt,85180436,english,favorite game of all time cant wait for the Ne...,1611373086,1611373086,True,0,0,0.000000,0,True,False,False,76561198065591528,33,1,23329.0,177.0,23329.0,1.611219e+09
20,20,292030,The Witcher 3: Wild Hunt,85179753,english,Why wouldn't you get this,1611371978,1611371978,True,0,0,0.000000,0,True,False,False,76561198996835044,131,2,8557.0,2004.0,8557.0,1.611371e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21747364,21747369,546560,Half-Life: Alyx,65645167,english,best game ever\n,1584986750,1584986750,True,0,0,0.000000,0,True,False,False,76561198817267089,86,8,1055.0,14.0,45.0,1.611138e+09
21747366,21747371,546560,Half-Life: Alyx,65645125,english,Holy balls,1584986704,1584986704,True,0,0,0.000000,0,True,False,False,76561197967033932,431,7,432.0,0.0,29.0,1.586628e+09
21747367,21747372,546560,Half-Life: Alyx,65645115,english,"A tripod thing bears down on you, it looks imm...",1584986693,1584986693,True,0,0,0.000000,0,False,False,False,76561198046495482,70,10,3296.0,15.0,24.0,1.611080e+09
21747368,21747373,546560,Half-Life: Alyx,65645100,english,Honestly this is the best vr game ever,1584986673,1584986673,True,0,0,0.474138,0,True,False,False,76561198176379749,79,17,2654.0,0.0,34.0,1.591634e+09


In [12]:
games_list = df['app_name'].value_counts().to_frame()
print(games_list.index.to_frame())

                                                                           0
Tom Clancy's Rainbow Six Siege                Tom Clancy's Rainbow Six Siege
Terraria                                                            Terraria
Garry's Mod                                                      Garry's Mod
Grand Theft Auto V                                        Grand Theft Auto V
PLAYERUNKNOWN'S BATTLEGROUNDS                  PLAYERUNKNOWN'S BATTLEGROUNDS
...                                                                      ...
DRAGON QUEST HEROES™ II                              DRAGON QUEST HEROES™ II
Senren＊Banka                                                    Senren＊Banka
Battle Royale Trainer                                  Battle Royale Trainer
三国群英传8 Heroes of the Three Kingdoms 8  三国群英传8 Heroes of the Three Kingdoms 8
Neon Hardcorps                                                Neon Hardcorps

[315 rows x 1 columns]


In [13]:
games_list_20 = games_list.sample(frac=0.03, random_state = 40)
games_list_20.shape

(9, 1)

In [14]:
#showing games in our sample of data
games_list_20_list = games_list_20.index.tolist()
df_20 = df.loc[(df['app_name'].isin(games_list_20_list))]

In [15]:
df_20.shape

(263815, 23)

In [16]:
df_20['app_name'].value_counts()

ARK: Survival Evolved           184576
Deep Rock Galactic               47002
Assassin's Creed Origins         17832
Avorion                           5056
FAR: Lone Sails                   3834
Titan Souls                       2633
Iconoclasts                       1492
SENRAN KAGURA ESTIVAL VERSUS      1119
Senren＊Banka                       271
Name: app_name, dtype: int64

In [19]:
#df_20_20 = df_20.sample(frac = 0.1)
df_20_20 = df_20.sample(frac = 0.1)
#df_20_20['recommended'].value_counts()
#df_20_20 = df_20

In [30]:
df_20_20['recommended'].value_counts()

True     21169
False     5213
Name: recommended, dtype: int64

In [44]:
df_20_20.groupby('written_during_early_access')['recommended'].value_counts()

written_during_early_access  recommended
False                        True           14313
                             False           2674
True                         True            6856
                             False           2539
Name: recommended, dtype: int64

In [37]:
prop = df_20_20.groupby('written_during_early_access')['recommended'].value_counts(normalize = True)
prop

written_during_early_access  recommended
False                        True           0.842586
                             False          0.157414
True                         True           0.729750
                             False          0.270250
Name: recommended, dtype: float64

In [38]:
n = df_20_20.groupby('written_during_early_access')['recommended'].count()
n

written_during_early_access
False    16987
True      9395
Name: recommended, dtype: int64

In [41]:
prop_early = prop[(True, False)]
prop_not_early = prop[(False, False)]
print(prop_early, prop_not_early)

0.2702501330494944 0.15741449343615707


In [42]:
n_early = n[True]
n_not_early = n[False]
print(n_early, n_not_early)

9395 16987


In [43]:
p_hat = (n_early * prop_early + n_not_early * prop_not_early) / (n_early + n_not_early)
std_error = np.sqrt(p_hat * (1-p_hat) / n_early + p_hat * (1-p_hat) / n_not_early)
z_score = (prop_early - prop_not_early) / std_error
print(z_score)

22.04006417590368


In [46]:
n_not_recommended = np.array([2674, 2539])
n_row = np.array([14313 + 2674, 6856 + 2539])
from statsmodels.stats.proportion import proportions_ztest
z_score, p_value = proportions_ztest(count = n_not_recommended, nobs = n_row, alternative = 'two-sided')
z_score, p_value

#Reject null hypothesis.
#There is a difference in proportion between not recommended reviews in early access and not recommended reviews outside
# of early access at a 1% level of significance. 

(-22.04006417590368, 1.1896856830193731e-107)

In [ ]:
df_20_20.review = df_20_20.review.astype(str)

In [ ]:
import string
import re

In [ ]:
#Convert text to lowercase and removing punctuation 

def remove_punct(txt):
    text_input = "".join([char for char in txt if char not in string.punctuation])
    return text_input

df_20_20['token_review'] = df_20_20['review'].apply(lambda x: remove_punct(x.lower()))

In [ ]:
#remove numbers

def remove_digits(txt):
    return re.sub(r' \d+', '', str(txt))

df_20_20['token_review1'] = df_20_20['token_review'].apply(lambda x: remove_digits(x))

In [ ]:
#remove non alphabet characters

def remove_chars(txt):
    return re.sub("[^a-zA-Z]+", ' ', txt)

df_20_20['token_review2'] = df_20_20['token_review1'].apply(lambda x: remove_chars(x))

In [ ]:
#Convert reviews into tokens

def tokenize(txt):
    tokens = re.split('\W+', txt)
    return tokens

df_20_20['token_review3'] = df_20_20['token_review2'].apply(lambda x: tokenize(x))

In [ ]:
#remove stopwords

stopword = nltk.corpus.stopwords.words('english')

def rem_stopwords(txt):
    text = [word for word in txt if word not in stopword]
    return text

df_20_20['token_review4'] = df_20_20['token_review3'].apply(lambda x: rem_stopwords(x))
    

In [ ]:
#convert words to base form 
wn = nltk.WordNetLemmatizer()

def lemmatizer(txt):
    text = [wn.lemmatize(word) for word in txt]
    return text

df_20_20['token_review5']=df_20_20['token_review4'].apply(lambda x: lemmatizer(x))

In [ ]:
df_20_20['reviews_fin'] = df_20_20['token_review5'].apply(lambda x:' '.join(x))
df_20_20['rec_var'] = df_20_20['recommended'].apply(lambda x: 0 if x==1 else 1)

In [ ]:
pos_reviews = df_20_20[df_20_20['rec_var']== 0] #majority variable positive reviews
neg_reviews = df_20_20[df_20_20['rec_var']== 1] #minority variable negative reviews

In [ ]:
print(pos_reviews['review'].count(),
    neg_reviews['review'].count())

In [ ]:
from nltk.probability import FreqDist

pos_rev_words = pos_reviews['token_review5']
pos_words = []
for reviews in pos_rev_words: 
    pos_words += reviews
    
freqdist_pos = FreqDist(pos_words)
freqdist_pos

In [ ]:
neg_rev_words = neg_reviews['token_review5']
neg_words = []
for reviews in neg_rev_words: 
    neg_words += reviews
    
freqdist_neg = FreqDist(neg_words)
freqdist_neg

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(width = 800, height = 800,
                     background_color = 'white',
                     stopwords = stopword,
                     min_font_size = 10).generate_from_frequencies(freqdist_pos)

plt.figure(figsize = (10,10), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('postive.png')
plt.show


In [ ]:
wordcloud = WordCloud(width = 800, height = 800,
                     background_color = 'white',
                     stopwords = stopword,
                     min_font_size = 10).generate_from_frequencies(freqdist_neg)

plt.figure(figsize = (10,10), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('negative.png')
plt.show


In [ ]:
(pd.Series(nltk.ngrams(pos_words, 2)).value_counts())[:10]

In [ ]:
(pd.Series(nltk.ngrams(pos_words, 3)).value_counts())[:10]

In [ ]:
(pd.Series(nltk.ngrams(neg_words, 2)).value_counts())[:10]

In [ ]:
(pd.Series(nltk.ngrams(neg_words, 3)).value_counts())[:10]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
df_x = df_20_20['reviews_fin']
y = df_20_20['rec_var']
X_train, X_test, y_train, y_test = train_test_split(df_x, y, 
                                                    test_size = 0.2, random_state = 42)


In [ ]:
import xgboost

In [ ]:
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_features.shape, test_vectors.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from time import time
from pprint import pprint
import logging
from sklearn.metrics import recall_score, roc_auc_score, make_scorer, roc_curve

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier

cv = KFold(n_splits=5)


pipe_multi = Pipeline([('vect', TfidfVectorizer()),
                      ('clf', MultinomialNB())])

pipe_rf = Pipeline([('vect', TfidfVectorizer()),
                      ('clf', RandomForestClassifier(random_state = 42))])


grid_params_multi = {'vect__ngram_range': [(1,1), (2,2), (3,3)]
                     }

grid_params_rf = {'vect__ngram_range': [(1,1), (2,2), (3,3)],
                  'clf__max_depth': [10, 20, 50, 100],
                  'clf__n_estimators': [200, 400, 600, 800]
                     }

multiNB = GridSearchCV(estimator = pipe_multi,
                      param_grid = grid_params_multi,
                      scoring = 'roc_auc',
                      cv = cv, 
                       n_jobs = -1)

rf = GridSearchCV(estimator = pipe_rf,
                  param_grid = grid_params_rf,
                  scoring = 'roc_auc',
                  cv = cv, 
                  n_jobs = -1)

grids = [multiNB, rf] #rf

grid_dict = {0: 'Naive Bayes',
            1: 'Random Forest'}

print('Performing Model Optimizations')
best_auc = 0.0
best_clf = 0
best_gs = ''

for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    gs.fit(X_train, y_train)
    print('Best params are : %s' % gs.best_params_)
    # Best training data recall score
    print('Best training AUC score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    y_pred_proba = gs.predict_proba(X_test)[:,1]
    # Test data accuracy of model with best params
    print('Test set AUC score for best params: %.3f ' % roc_auc_score(y_test, y_pred_proba))
    #Plot ROC curve for each estimator
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test,y_pred_proba)
    plt.plot([0,1],[0,1], linestyle ='--')
    plt.plot(fpr, tpr, label = 'auc=%.3f' % auc)
    plt.title(label = 'ROC Curve %s' % grid_dict[idx])
    name = grid_dict[idx]
    plt.savefig(name + ".png")
    plt.show()
    # Track best (highest test auc) model
    if roc_auc_score(y_test, y_pred_proba) > best_auc:
        best_auc = roc_auc_score(y_test, y_pred_proba)
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set AUC: %s' % grid_dict[best_clf])




In [ ]:
print(rf.best_params_)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range = (1,1))
X_train_vec = vectorizer.fit_transform(X_train)
X_train_vec_df = pd.DataFrame(X_train_vec.toarray())
X_test_vec = vectorizer.transform(X_test)
X_test_vec_df = pd.DataFrame(X_test_vec.toarray())


In [ ]:
words = vectorizer.get_feature_names()

In [ ]:
X_train_vec_df.columns = words
X_test_vec_df.columns = words

In [ ]:
clf = RandomForestClassifier(random_state = 42, max_depth= 100, n_estimators= 800)
clf_fit = clf.fit(X_train_vec_df, y_train)


In [ ]:
import shap

In [ ]:
from scipy import interpolate
from sklearn import metrics

In [ ]:
explainer = shap.TreeExplainer(clf_fit, X_train_vec_df, feature_names = words)
shap_values = explainer(X_test_vec_df)
print(shap_values.values.shape)

In [ ]:
import copy

shap.initjs()
shap_values1 = copy.deepcopy(shap_values)
shap_values1.values = shap_values1.values[:,:,1]
shap_values1.base_values = shap_values1.base_values[:,1]

shap.plots.beeswarm(shap_values1, max_display = 20, show= False)
plt.savefig('beeswarm.png', bbox_inches="tight")

In [ ]:
shap.initjs()

ind = 48
print(X_test.iloc[ind], y_test.iloc[ind])

In [ ]:
shap.plots.waterfall(shap_values[ind,:,1], show = False)
plt.savefig('waterfall.png', bbox_inches="tight")

In [ ]:
shap.plots.scatter(shap_values[:,words.index('story'),1], show = False)
plt.savefig('scatter.png', bbox_inches="tight")

In [ ]:
y_test.value_counts()

In [ ]:
feature_scores = pd.Series(clf_fit.feature_importances_,
                          index = words).sort_values(ascending=False)
feature_scores[:10]


In [ ]:
feat_imp = feature_scores.nlargest(10).sort_values(ascending=True)
feat_imp.plot(kind = 'barh', figsize = (10,10))
plt.savefig('feat_imp.png', bbox_inches="tight")

In [ ]:
from sklearn.metrics import precision_recall_curve

y_pred_proba = clf_fit.predict_proba(X_test_vec_df)[:,1]
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold = %f, F-Score = %.3f' % (thresholds[ix], fscore[ix]))
plt.plot(recall, precision, label = 'auc=%.3f' % auc, marker = '.')
plt.plot(recall[ix], precision[ix], marker = 'o', color = 'black', label = 'Best')
plt.title(label = 'Precision Recall Curve')
plt.savefig('prec_recall.png', bbox_inches="tight")
plt.show()
optimal_thresh = thresholds[ix]
optimal_prec = precision[ix]

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = (y_pred_proba > optimal_thresh)
conf_mat = confusion_matrix(y_test, y_pred.astype(int), labels = [1,0])
conf_mat

In [ ]:
print(X_test.shape)
y_pred_df = y_pred.astype(int)
X_variables = pd.DataFrame(X_test)
X_variables['predictions'] = y_pred_df

In [ ]:
negative_reviews = X_variables[X_variables['predictions'] == 1 ]
negative_reviews.head(10)


In [ ]:
negative_reviews.count()

In [ ]:
X_variables.iloc[ind]

## Unused Code For Now

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

In [ ]:
precision.shape, recall.shape, thresholds.shape

In [ ]:
precision = np.delete(precision, -1)
recall = np.delete(recall, -1)

precision.shape, recall.shape, thresholds.shape

In [ ]:
#locate recall from selected precision

precision_score = optimal_prec
find_recall = interpolate.interp1d(precision, recall)
find_thresh = interpolate.interp1d(precision, thresholds)
thresh_value = find_thresh(precision_score)
recall_score = find_recall(precision_score)
recall_score, thresh_value

In [ ]:
plt.plot(recall, precision, label = 'auc=%.3f' % auc, marker = '.')
plt.plot(recall_score, precision_score, marker = 'o', color = 'black', label = 'Best')
plt.title(label = 'Precision Recall Curve')
plt.show()


In [ ]:
tpr_opt = find_tpr(thresh_value)
fpr_opt = find_fpr(thresh_value)
print(tpr_opt, fpr_opt, thresh_value)

In [ ]:
#locating optimal point
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

find_tpr = interpolate.interp1d(thresholds, tpr) #plug in thresholds for tpr
find_fpr = interpolate.interp1d(thresholds, fpr) #plug in thresh for fpr
thresh_look = interpolate.interp1d(tpr, thresholds) #plug in tpr for thresholds


In [ ]:
ttt = thresh_look(0.6) #threshold value for value of tpr
ttt

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
auc = roc_auc_score(y_test,y_pred_proba)
plt.plot([0,1],[0,1], linestyle ='--')
plt.plot(fpr, tpr, label = 'auc=%.3f' % auc, marker = '.')
plt.plot(fpr_opt, tpr_opt, marker = 'o', color = 'black')
plt.title(label = 'ROC Curve Random Forest Classifier')
plt.show()

In [ ]:
y_pred_opt = (y_pred_proba > ttt)
conf_mat_opt = confusion_matrix(y_test, y_pred_opt, labels = [0,1])
conf_mat_opt

In [ ]:
#find best true negative (tn) results
tn = 7000
best_thresh = 0
for t in high_thresh:
    y_pred_opt = (y_pred_proba > t)
    conf_mat_opt = confusion_matrix(y_test, y_pred_opt, labels = [0,1])
    if conf_mat_opt[1][0] > tn:
        best_thresh = t
        tn = conf_mat_opt[1][0]
        conf_mat_best = conf_mat_opt

best_thresh, tn, conf_mat_best

In [ ]:
high_thresh = thresholds[thresholds > 0.5]
high_thresh

In [ ]:
from sklearn.metrics import accuracy_score

#accuracy prior to choosing threshold
accuracy = accuracy_score(y_test, y_pred_proba.astype('int'))
print(accuracy)

In [ ]:
y_pred = y_pred_proba
y_pred = (y_pred >= opt_thresh).astype('int')

accuracy_thresh = accuracy_score(y_test, y_pred)
print(accuracy_thresh)

In [ ]:
plt.plot(recall, precision, label = 'auc=%.3f' % auc, marker = '.')
plt.plot(0.9, prec_point, marker = 'o', color = 'black', label = 'Best')
plt.title(label = 'Precision Recall Curve')
plt.show()
